# SNN that detects Network bursts
This notebook is a simple example of how to use a Spiking Neural Network (SNN) to detect network bursts in a network of 5 neurons (channels)

## Definition of a network burst
A network burst is a sequence of spikes that occur in a short time window in a neural population. The definition of a network burst is not unique and depends on the context. 

In this notebook, we will **consider a channel burst a neuronal activity where 4 spikes occur within 20ms in the same channel**.

In this notebook, we will **consider a network burst a neuronal activity where 10 channels spike in a 20ms time frame**.

### Check WD (change if necessary) and file loading

In [46]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/network_bursts" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/network_bursts"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/network_bursts


In [47]:
from lava.proc.lif.process import LIF, LIFRefractory
from lava.proc.dense.process import Dense
import numpy as np

LIF?

Init signature: LIF(*args, **kwargs)
Docstring:     
Leaky-Integrate-and-Fire (LIF) neural Process.

LIF dynamics abstracts to:
u[t] = u[t-1] * (1-du) + a_in         # neuron current
v[t] = v[t-1] * (1-dv) + u[t] + bias  # neuron voltage
s_out = v[t] > vth                    # spike if threshold is exceeded
v[t] = 0                              # reset at spike

Parameters
----------
shape : tuple(int)
    Number and topology of LIF neurons.
u : float, list, numpy.ndarray, optional
    Initial value of the neurons' current.
v : float, list, numpy.ndarray, optional
    Initial value of the neurons' voltage (membrane potential).
du : float, optional
    Inverse of decay time-constant for current decay. Currently, only a
    single decay can be set for the entire population of neurons.
dv : float, optional
    Inverse of decay time-constant for voltage decay. Currently, only a
    single decay can be set for the entire population of neurons.
bias_mant : float, list, numpy.ndarray, optiona

## Define the Architecture of the Network

In [48]:
# Define the number of neurons in each LIF Layer
n1 = 251
n2 = 1

## Choose the LIF Models to use

In [49]:
import numpy as np

# -- LIF Parameters --
# Fixed
v_th = 1
v_init = 0
C_IBI = 20    # Channel Inter-Burst Interval (ms)
N_IBI = 40    # Network Inter-Burst Interval (ms)
# Tunable
du1 = 0.45
dv1 = 0.05
du2 = 0.9
dv2 = 0.05
use_refractory = True
refrac_period1 = C_IBI  # Should it be the same for both layers?
refrac_period2 = N_IBI

# Scale the weights
weights_scale_input = 0.2
weights_scale_middle = 0.15

# Simulation Parameters
init_offset = 0  # 1000                  
virtual_time_step_interval = 1  
num_steps = 1 * (10**5)  # 30000    # 1000      # 3000  # 26500     # TODO: Check the number of steps to run the simulation for

# Input File
file_path =  "../lab_data/lab_data_all_channels.csv"   # "./data/custom_4spikes_20ms_ch1.csv"       # "../lab_data/lab_data_1-8channels.csv"

# RUN CONFIGURATION
LIGHT_RUN = False    # Only monitor the output state

### Create the LIF Processes

In [50]:
# Create Processes
lif1 = LIF(
    shape=(n1,),  # There are 2 neurons
    vth=v_th,  # TODO: Verify these initial values
    v=v_init,
    dv=dv1,    # Inverse of decay time-constant for voltage decay
    du=du1,  # Inverse of decay time-constant for current decay
    bias_mant=0,
    bias_exp=0,
    name="lif1"
)

lif2 = LIF(
    shape=(n2,),  # There is 1 neuron
    vth=v_th,  # TODO: Verify these initial values
    v=v_init,
    dv=dv2,    # Inverse of decay time-constant for voltage decay
    du=du2,  # Inverse of decay time-constant for current decay
    bias_mant=0,
    bias_exp=0,
    name="lif2"
)

In [51]:
refrac_lif1 = LIFRefractory(
    shape=(n1,),  # There are 2 neurons
    vth=v_th,  # TODO: Verify these initial values
    v=v_init,
    dv=dv1,    # Inverse of decay time-constant for voltage decay
    du=du1,  # Inverse of decay time-constant for current decay
    bias_mant=0,
    bias_exp=0,
    refractory_period=refrac_period1,
    name="lif1"
)

refrac_lif2 = LIFRefractory(
    shape=(n2,),  # There are 2 neurons
    vth=v_th,  # TODO: Verify these initial values
    v=v_init,
    dv=dv2,    # Inverse of decay time-constant for voltage decay
    du=du2,  # Inverse of decay time-constant for current decay
    bias_mant=0,
    bias_exp=0,
    refractory_period=refrac_period2,
    name="lif2"
)

### Choose the Selected LIF

In [52]:
selected_lif1 = refrac_lif1 if use_refractory else lif1
selected_lif2 = refrac_lif2 if use_refractory else lif2

## Create the Custom Input Layer

In [53]:
from utils.input import read_spike_events

# Call the function to read the spike events
spike_events = read_spike_events(file_path)
print("Spike events: ", spike_events.shape, spike_events[:10])

Spike events:  (40020, 2) [[ 99.5 229. ]
 [303.6   7. ]
 [502.5 229. ]
 [510.6  71. ]
 [528.   54. ]
 [540.9   7. ]
 [589.3 225. ]
 [631.6 100. ]
 [633.8 100. ]
 [758.3 229. ]]


In [54]:
# Find the max channel idx
max_channel_idx = int(np.max(spike_events[:, 1]))
print("Max Channel Idx: ", max_channel_idx)

Max Channel Idx:  251


### Map the input channels to the corresponding indexes in the input layer
Since the input channels in the input file may be of any number, we need to map the input channels to the corresponding indexes in the input layer. This is done by the `channel_map` dictionary.

In [55]:
# Map the channels of the input file to the respective index in the output list of SpikeEventGen
# channel_map = {ch:idx for idx, ch in enumerate(range(45, 55, 1))}

# Create a channel map based on the max channel idx
channel_map = {channel: idx for idx, channel in enumerate(range(1, max_channel_idx+1))}

relevant_channels = list(channel_map.keys())

print("Channel Map: ", channel_map)
print("Number of channels: ", len(channel_map))
print("Relevant Channels: ", relevant_channels)

Channel Map:  {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121

## Generate the Ground Truth

### Filter the `spike_events` belonging to a relevant channel

In [56]:
from utils.io import preview_np_array

rel_spike_events = list(filter(lambda x: x[1] in relevant_channels, spike_events))
rel_spike_events = np.array(rel_spike_events)

preview_np_array(rel_spike_events, "Relevant Spike Events")

Relevant Spike Events Shape: (40014, 2).
Preview: [[9.950000e+01 2.290000e+02]
 [3.036000e+02 7.000000e+00]
 [5.025000e+02 2.290000e+02]
 [5.106000e+02 7.100000e+01]
 [5.280000e+02 5.400000e+01]
 ...
 [2.999121e+05 1.000000e+00]
 [2.999170e+05 1.950000e+02]
 [2.999202e+05 5.400000e+01]
 [2.999223e+05 4.500000e+01]
 [2.999228e+05 5.400000e+01]]


### 1) Find the Channel Bursts first

In [57]:
from ground_truth import find_channel_bursts

# Generate the Ground Truth -> Find the network bursts according to the conditions specified
CH_CAUSALITY_WINDOW = C_IBI

# Parameters to Calculate Channel Bursts
num_spikes_to_burst = 4
max_burst_duration = CH_CAUSALITY_WINDOW
min_inter_burst_interval = CH_CAUSALITY_WINDOW
# Calculate the Channel Bursts
ch_bursts, ch_bursts_detailed = find_channel_bursts(rel_spike_events, num_spikes_to_burst, max_burst_duration, min_inter_burst_interval, verbose=False)

print(f"CBs detected {sum([len(ch_bursts_detailed[channel]) for channel in ch_bursts_detailed])} bursts")
print("Channel Bursts: ", ch_bursts)
print("Channel Bursts Detailed: ", ch_bursts_detailed)

CBs detected 3701 bursts
Channel Bursts:  {195.0: [1430.5, 2794.3, 2874.8, 4192.8, 5013.7, 5879.0, 5901.8, 5912.3, 5944.0, 5954.0, 5988.900000000001, 6001.8, 9848.7, 18772.7, 18806.7, 21585.2, 22081.4, 22176.9, 22208.7, 22220.3, 26736.5, 29603.5, 30794.9, 30814.8, 30844.7, 33297.5, 33408.8, 33441.6, 33451.0, 33481.6, 33508.5, 36258.1, 37081.4, 41398.2, 43103.1, 44341.5, 45168.0, 45201.7, 46132.9, 46789.1, 47369.0, 47413.6, 47447.6, 47457.3, 47492.5, 47505.7, 47536.6, 47549.1, 50238.5, 53910.9, 54706.4, 55153.5, 56964.5, 58732.1, 58762.7, 58772.0, 58803.3, 61612.4, 63493.9, 65648.0, 65682.1, 65691.7, 65724.2, 67098.8, 67996.3, 68151.7, 69477.4, 69517.1, 72148.3, 74914.2, 77350.0, 78235.90000000001, 78464.8, 79529.40000000001, 79557.40000000001, 79568.90000000001, 79599.90000000001, 81339.8, 81370.90000000001, 81385.40000000001, 82535.0, 84119.5, 84972.40000000001, 86599.90000000001, 87508.40000000001, 87536.7, 87545.7, 87576.40000000001, 87599.2, 87609.0, 87642.0, 88376.0, 89218.9000000

### 2) Find the Network Bursts using the Channel Bursts as the input spikes

In [58]:
# Convert the Channel Bursts to the same format as the spike_events
ch_burst_events = []

for ch, burst_times in ch_bursts.items():
    for time in burst_times:
        ch_burst_events.append([time, ch])

ch_burst_events = np.array(ch_burst_events)
# Sort the channel burst events by time
ch_burst_events = ch_burst_events[ch_burst_events[:, 0].argsort()]

preview_np_array(ch_burst_events, "ch_burst_events", edge_items=40)

ch_burst_events Shape: (3701, 2).
Preview: [[1.430500e+03 1.950000e+02]
 [2.794300e+03 1.950000e+02]
 [2.796500e+03 2.290000e+02]
 [2.802100e+03 2.400000e+02]
 [2.874800e+03 1.950000e+02]
 [2.882000e+03 2.290000e+02]
 [2.887100e+03 1.000000e+00]
 [2.892200e+03 2.400000e+02]
 [2.907000e+03 5.400000e+01]
 [2.907700e+03 4.500000e+01]
 [2.915200e+03 5.200000e+01]
 [4.192800e+03 1.950000e+02]
 [4.999700e+03 5.400000e+01]
 [5.002400e+03 4.500000e+01]
 [5.008900e+03 1.000000e+02]
 [5.010500e+03 7.100000e+01]
 [5.010600e+03 1.000000e+00]
 [5.012600e+03 1.250000e+02]
 [5.012800e+03 2.400000e+02]
 [5.012900e+03 1.930000e+02]
 [5.013700e+03 1.950000e+02]
 [5.014900e+03 1.820000e+02]
 [5.015500e+03 2.300000e+02]
 [5.016300e+03 6.500000e+01]
 [5.019300e+03 2.290000e+02]
 [5.023600e+03 8.700000e+01]
 [5.026300e+03 1.710000e+02]
 [5.033500e+03 4.500000e+01]
 [5.034200e+03 5.400000e+01]
 [5.879000e+03 1.950000e+02]
 [5.883900e+03 1.500000e+02]
 [5.884100e+03 2.290000e+02]
 [5.884900e+03 1.710000e+02]


In [59]:
from ground_truth import find_net_bursts

# Generate the Ground Truth -> Find the network bursts according to the conditions specified
NET_CAUSALITY_WINDOW = C_IBI

# Parameters to Calculate Network Bursts
num_spikes_to_burst = 10
max_burst_duration = NET_CAUSALITY_WINDOW
min_inter_burst_interval = N_IBI
# Calculate the Network Bursts
gt, gt_detailed = find_net_bursts(ch_burst_events, num_spikes_to_burst, max_burst_duration, min_inter_burst_interval, verbose=False)

print(f"NBs detected {len(gt)} bursts")
print("Network Bursts: ", gt)
print("Network Bursts Detailed: ", gt_detailed)

NBs detected 100 bursts
Network Bursts:  [5014.900000000001, 5892.7, 5953.3, 6015.0, 22079.4, 22173.8, 29608.8, 30834.0, 33408.8, 33468.2, 45174.7, 47423.4, 47483.8, 47545.1, 50250.1, 55152.9, 58730.1, 58791.0, 61628.2, 65669.5, 68151.7, 69420.5, 69477.1, 79559.6, 81342.7, 86607.2, 87508.40000000001, 87571.2, 102442.3, 104110.1, 105361.8, 105422.5, 116751.0, 118933.3, 121314.6, 121375.0, 128058.8, 131063.2, 132100.4, 133987.7, 135884.5, 137118.6, 138998.3, 139058.8, 140672.6, 141381.30000000002, 141444.0, 141499.7, 157727.30000000002, 161414.6, 166125.9, 167626.1, 168899.1, 169932.5, 170926.1, 172735.0, 172798.5, 172858.7, 195657.5, 196671.3, 197515.5, 197576.0, 199632.3, 201699.0, 203126.3, 208880.4, 210311.1, 211420.3, 212252.0, 213621.6, 215157.8, 216497.2, 217260.1, 218195.3, 219661.6, 221411.2, 221469.8, 229605.7, 231904.2, 235940.7, 239237.7, 241789.4, 246964.6, 247990.7, 251354.3, 253967.0, 255725.4, 256805.4, 257885.1, 259242.8, 259811.7, 260658.9, 261306.2, 264394.3, 265757.3,

Crop the ground truth according to the simulation time and selected channels

In [60]:
gt_detailed_cropped = gt_detailed.copy()
gt_detailed_cropped = [burst_info for burst_info in gt_detailed_cropped if (
    burst_info[0] >= init_offset and
    burst_info[0] < init_offset + num_steps*virtual_time_step_interval
    )
]

print(f"Ground Truth Detailed Cropped has {len(gt_detailed_cropped)} bursts")
print("Ground Truth Detailed Cropped: ", gt_detailed_cropped)

Ground Truth Detailed Cropped has 28 bursts
Ground Truth Detailed Cropped:  [(5014.900000000001, [54.0, 45.0, 100.0, 71.0, 1.0, 125.0, 240.0, 193.0, 195.0, 182.0]), (5892.7, [195.0, 150.0, 229.0, 171.0, 189.0, 182.0, 193.0, 240.0, 220.0, 100.0]), (5953.3, [86.0, 71.0, 54.0, 1.0, 87.0, 93.0, 65.0, 240.0, 189.0, 220.0, 125.0, 89.0, 195.0, 49.0, 7.0, 4.0, 45.0, 230.0]), (6015.0, [65.0, 195.0, 45.0, 71.0, 229.0, 100.0, 193.0, 182.0, 54.0, 89.0, 125.0]), (22079.4, [100.0, 54.0, 87.0, 65.0, 71.0, 45.0, 1.0, 86.0, 150.0, 107.0]), (22173.8, [100.0, 83.0, 86.0, 87.0, 89.0, 54.0, 97.0, 71.0, 93.0, 125.0]), (29608.8, [150.0, 100.0, 195.0, 71.0, 93.0, 125.0, 182.0, 65.0, 1.0, 193.0]), (30834.0, [195.0, 229.0, 240.0, 150.0, 45.0, 171.0, 100.0, 182.0, 189.0, 1.0]), (33408.8, [100.0, 97.0, 93.0, 71.0, 150.0, 182.0, 87.0, 86.0, 125.0, 195.0]), (33468.2, [87.0, 86.0, 195.0, 193.0, 65.0, 1.0, 45.0, 54.0, 4.0, 240.0, 229.0, 220.0, 93.0, 71.0]), (45174.7, [150.0, 100.0, 195.0, 65.0, 125.0, 71.0, 54.0, 87.

In [61]:
# Crop the ground truth according to the simulation time
gt_cropped = gt.copy()
gt_cropped = [spike_time for spike_time in gt_cropped if (
    spike_time >= init_offset and
    spike_time < init_offset + num_steps*virtual_time_step_interval
    )
]

print(f"Ground Truth Cropped has {len(gt_cropped)} bursts")
print("Ground Truth Cropped: ", gt_cropped)

Ground Truth Cropped has 28 bursts
Ground Truth Cropped:  [5014.900000000001, 5892.7, 5953.3, 6015.0, 22079.4, 22173.8, 29608.8, 30834.0, 33408.8, 33468.2, 45174.7, 47423.4, 47483.8, 47545.1, 50250.1, 55152.9, 58730.1, 58791.0, 61628.2, 65669.5, 68151.7, 69420.5, 69477.1, 79559.6, 81342.7, 86607.2, 87508.40000000001, 87571.2]


### Create the `SpikeEventGenerator` object 

In [62]:
from utils.spike_event_gen import SpikeEventGen

# Create the Input Process
spike_event_gen = SpikeEventGen(shape=(n1,), spike_events=rel_spike_events, name="CustomInput", channel_map=channel_map,
                            virtual_time_step_interval=virtual_time_step_interval, init_offset=init_offset)

# TODO: Check the channels being used and alter the GT to only view those channnels

### Create the Dense Layers

In [63]:
# Create Dense Process to connect the input layer and LIF1
# create weights of the dense layer
dense_weights_input = np.eye(N=n1, M=n1)
# multiply the weights of the Dense layer by a constant
dense_weights_input *= weights_scale_input
dense_input = Dense(
    weights=np.array(dense_weights_input), 
    name="DenseInput"
)


# Create Dense Process to connect LIF1 and LIF2
# create weights of the dense layer connecting LIF1 and LIF2
dense_weights_middle = np.ones(shape=(n2, n1))

# multiply the weights of the Dense layer by a constant
dense_weights_middle *= weights_scale_middle

# Create Dense Process to connect the two LIF layers
dense_middle = Dense(
    shape=(n1, n2),  # There are 2 neurons in the first layer and 1 in the second
    weights=np.array(dense_weights_middle),
    name="Dense_LIF1-2"
)

## Connect the Layers

In [64]:
# Connect the SpikeEventGen to the Dense Layer
spike_event_gen.s_out.connect(dense_input.s_in)

# Connect the Dense_Input to the LIF1 Layer
dense_input.a_out.connect(selected_lif1.a_in)

# Connect the LIF1 Layer to the Dense Layer
selected_lif1.s_out.connect(dense_middle.s_in)   # Connect the output of the first LIF layer to the Dense Layer
# Connect the Dense Layer to the LIF2 Layer
dense_middle.a_out.connect(selected_lif2.a_in)   # Connect the output of the Dense Layer to the second LIF Layer

### Take a look at the connections in the Input Layer

In [65]:
for proc in [spike_event_gen, dense_input, selected_lif1, dense_middle, selected_lif2]:
    for port in proc.in_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")
    for port in proc.out_ports:
        print(f"Proc: {proc.name:<5} Port Name: {port.name:<5} Size: {port.size}")

Proc: CustomInput Port Name: s_out Size: 251
Proc: DenseInput Port Name: s_in  Size: 251
Proc: DenseInput Port Name: a_out Size: 251
Proc: lif1  Port Name: a_in  Size: 251
Proc: lif1  Port Name: s_out Size: 251
Proc: Dense_LIF1-2 Port Name: s_in  Size: 251
Proc: Dense_LIF1-2 Port Name: a_out Size: 1
Proc: lif2  Port Name: a_in  Size: 1
Proc: lif2  Port Name: s_out Size: 1


### Look at the weights of the Dense Layers

In [66]:
# Weights of the Input Dense Layer
dense_input.weights.get()

array([[0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , ..., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , ..., 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
   

In [67]:
# Weights of the Dense Layer between LIF1 and LIF2
dense_middle.weights.get()

array([[0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, ..., 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15,
        0.15, 0.15, 0.15, 0.15]])

### Record Internal Vars over time
To record the evolution of the internal variables over time, we need a `Monitor`. For this example, we want to record the membrane potential of the `LIF` Layer, hence we need 1 `Monitors`.

We can define the `Var` that a `Monitor` should record, as well as the recording duration, using the `probe` function

In [68]:
from lava.proc.monitor.process import Monitor

monitor_lif1_v = Monitor()
monitor_lif1_u = Monitor()
monitor_lif2_v = Monitor()
monitor_lif2_u = Monitor()

# Connect the monitors to the variables we want to monitor
if not LIGHT_RUN:
    monitor_lif1_v.probe(selected_lif1.v, num_steps)
    monitor_lif1_u.probe(selected_lif1.u, num_steps)
monitor_lif2_v.probe(selected_lif2.v, num_steps)
monitor_lif2_u.probe(selected_lif2.u, num_steps)

## Execution
Now that we have defined the network, we can execute it. We will use the `run` function to execute the network.

### Run Configuration and Conditions

In [69]:
from lava.magma.core.run_conditions import RunContinuous, RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg

# run_condition = RunContinuous()   # TODO: Change to this one
run_condition = RunSteps(num_steps=num_steps)
run_cfg = Loihi1SimCfg(select_tag="floating_pt")   # TODO: Check why we need this select_tag="floating_pt"

### Execute

In [70]:
selected_lif1.run(condition=run_condition, run_cfg=run_cfg)

/home/monkin/Desktop/feup/thesis/thesis-lava/src/lava/magma/compiler/compiler_graphs.py:895: UserWarning: Cannot import module '<module 'snn' from '/home/monkin/Desktop/feup/thesis/thesis-lava/src/utils/snn.py'>' when searching ProcessModels for Process 'SpikeEventGen'.
  warnings.warn(


### Retrieve recorded data

In [71]:
if not LIGHT_RUN:
    data_lif1_v = monitor_lif1_v.get_data()
    data_lif1_u = monitor_lif1_u.get_data()

    data_lif1 = data_lif1_v.copy()
    data_lif1["lif1"]["u"] = data_lif1_u["lif1"]["u"]   # Merge the dictionaries to contain both voltage and current

data_lif2_v = monitor_lif2_v.get_data()
data_lif2_u = monitor_lif2_u.get_data()

data_lif2 = data_lif2_v.copy()
data_lif2["lif2"]["u"] = data_lif2_u["lif2"]["u"]   # Merge the dictionaries to contain both voltage and current


In [72]:
# print("data_lif1:", data_lif1)

In [73]:
# print("data_lif2:", data_lif2)

In [74]:
# Check the shape to verify if it is printing the voltage for every step
if not LIGHT_RUN:
    preview_np_array(data_lif1['lif1']['v'], "LIF1 V:", edge_items=3)    # Indeed, there are 300 values (same as the number of steps we ran the simulation for)

preview_np_array(data_lif2['lif2']['v'], "LIF2 V:", edge_items=3)   

LIF1 V: Shape: (100000, 251).
Preview: [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 ...
 [7.10578565e-003 0.00000000e+000 1.43279037e-322 ... 3.41687622e-279
  1.43279037e-322 2.12984586e-217]
 [6.75049637e-003 0.00000000e+000 1.43279037e-322 ... 3.24603241e-279
  1.43279037e-322 2.02335357e-217]
 [6.41297155e-003 0.00000000e+000 1.43279037e-322 ... 3.08373079e-279
  1.43279037e-322 1.92218589e-217]]
LIF2 V: Shape: (100000, 1).
Preview: [[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 ...
 [4.36552799e-13]
 [4.14725159e-13]
 [3.93988901e-13]]


## Find the timesteps where the network bursts occur

In [75]:
from utils.data_analysis import find_spike_times

if not LIGHT_RUN:
    spike_times_lif1 = find_spike_times(data_lif1['lif1']['v'], data_lif1['lif1']['u'])
    preview_np_array(spike_times_lif1, "Spike Times LIF1", edge_items=5)


# Call the find_spike_times util function that detects the spikes in a voltage array
spike_times_lif2 = find_spike_times(data_lif2['lif2']['v'], data_lif2['lif2']['u'])

preview_np_array(spike_times_lif2, "Spike Times LIF2", edge_items=5)

Spike Times LIF1 Shape: (1218, 2).
Preview: [[ 1431   194]
 [ 1436     0]
 [ 2793   194]
 [ 2794   228]
 [ 2796   149]
 ...
 [97197   228]
 [99444    99]
 [99445    88]
 [99451    53]
 [99454    70]]
Spike Times LIF2 Shape: (36, 2).
Preview: [[ 5012     0]
 [ 5891     0]
 [ 5944     0]
 [ 6004     0]
 [20757     0]
 ...
 [84978     0]
 [86604     0]
 [87507     0]
 [87553     0]
 [87602     0]]


## View the Voltage and Current dynamics with an interactive plot

Grab the data from the recorded variables

In [76]:
# LIF1 variables
if not LIGHT_RUN:
    lif1_voltage_vals = data_lif1['lif1']['v']
    lif1_current_vals = data_lif1['lif1']['u']

    print("lif1 voltage shape:", len(lif1_voltage_vals))
    # print("voltage head: ", lif1_voltage_vals[:10])


# LIF2 variables
lif2_voltage_vals = data_lif2['lif2']['v']
lif2_current_vals = data_lif2['lif2']['u']

print("lif2 voltage shape:", len(lif2_voltage_vals))

lif1 voltage shape: 100000
lif2 voltage shape: 100000


## Assemble the values to be plotted

In [95]:
chosen_channels = [0, 3, 5, 6, 40, 45, 59, 174]
num_chosen_ch = len(chosen_channels)

voltage_lif1_y_arrays = [ ( lif1_voltage_vals[:, chosen_ch], f"Neuron. {chosen_ch}" ) for chosen_ch in chosen_channels ]

print("Voltage Lif1 Y Arrays: ", voltage_lif1_y_arrays[:5])

Voltage Lif1 Y Arrays:  [(array([0.        , 0.        , 0.        , 0.        , 0.        , ...,
       0.00787345, 0.00747977, 0.00710579, 0.0067505 , 0.00641297]), 'Neuron. 0'), (array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, ..., 1.17497170e-259, 1.11622312e-259,
       1.06041196e-259, 1.00739136e-259, 9.57021796e-260]), 'Neuron. 3'), (array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, ..., 4.28938819e-74, 4.07491878e-74,
       3.87117284e-74, 3.67761420e-74, 3.49373349e-74]), 'Neuron. 5'), (array([0.        , 0.        , 0.        , 0.        , 0.        , ...,
       0.00402806, 0.00382666, 0.00363532, 0.00345356, 0.00328088]), 'Neuron. 6'), (array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, ..., 6.11098520e-277, 5.80543594e-277,
       5.51516414e-277, 5.23940594e-277, 4.97743564e-277]), 'Neuron. 40')]


In [96]:
current_lif1_y_arrays = [ ( lif1_current_vals[:, chosen_ch], f"Neuron. {chosen_ch}" ) for chosen_ch in chosen_channels ]
print("Current Lif1 Y Arrays: ", current_lif1_y_arrays[:5])

Current Lif1 Y Arrays:  [(array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, ..., 3.02855616e-25, 1.66570589e-25,
       9.16138239e-26, 5.03876031e-26, 2.77131817e-26]), 'Neuron. 0'), (array([0.e+000, 0.e+000, 0.e+000, 0.e+000, 0.e+000, ..., 5.e-324, 5.e-324,
       5.e-324, 5.e-324, 5.e-324]), 'Neuron. 3'), (array([0.e+000, 0.e+000, 0.e+000, 0.e+000, 0.e+000, ..., 5.e-324, 5.e-324,
       5.e-324, 5.e-324, 5.e-324]), 'Neuron. 5'), (array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, ..., 1.42810493e-25, 7.85457712e-26,
       4.32001742e-26, 2.37600958e-26, 1.30680527e-26]), 'Neuron. 6'), (array([0.e+000, 0.e+000, 0.e+000, 0.e+000, 0.e+000, ..., 5.e-324, 5.e-324,
       5.e-324, 5.e-324, 5.e-324]), 'Neuron. 40')]


In [114]:
from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

x = [val + init_offset for val in range(num_steps)]

v_y1 = [val[0] for val in lif2_voltage_vals]

# Create the plot
voltage_lif2_y_arrays = [(v_y1, "Ch. 1")]    # List of tuples containing the y values and the legend label
# Define the box annotation parameters
box_annotation_voltage = {
    "bottom": 0,
    "top": v_th,
    "left": 0,
    "right": num_steps,
    "fill_alpha": 0.03,
    "fill_color": "green"
}

# Create the LIF2 Voltage
voltage_lif2_plot = create_fig(
    title="LIF2 Voltage dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (V)',
    x=x, 
    y_arrays=voltage_lif2_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    box_annotation_params=box_annotation_voltage,
    y_range=Range1d(-0.05, 1.05),
)


# Create the LIF2 Current
u_y1 = [val[0] for val in lif2_current_vals]
current_lif2_y_arrays = [(u_y1, "Output")]    # List of tuples containing the y values and the legend label
current_lif2_plot = create_fig(
    title="LIF2 Current dynamics", 
    x_axis_label='time (ms)', 
    y_axis_label='Current (U)',
    x=x, 
    y_arrays=current_lif2_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    x_range=voltage_lif2_plot.x_range,    # Link the x-axis range to the voltage plot
)

# bplt.show(voltage_lif1_plot)

In [98]:
# Define the x and y values
if not LIGHT_RUN:
    # Define the box annotation parameters
    box_annotation_voltage = {
        "bottom": 0,
        "top": v_th,
        "left": 0,
        "right": num_steps,
        "fill_alpha": 0.03,
        "fill_color": "green"
    }

    # Create the LIF1 Voltage
    voltage_lif1_plot = create_fig(
        title="LIF1 Voltage dynamics", 
        x_axis_label='time (ms)', 
        y_axis_label='Voltage (V)',
        x=x, 
        y_arrays=voltage_lif1_y_arrays, 
        sizing_mode="stretch_both", 
        tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
        tooltips="Data point @x: @y",
        legend_location="top_right",
        legend_bg_fill_color="navy",
        legend_bg_fill_alpha=0.1,
        box_annotation_params=box_annotation_voltage,
        y_range=Range1d(-0.05, 1.05),
        x_range=voltage_lif2_plot.x_range,    # Link the x-axis range to the voltage plot
    )

    current_lif1_plot = create_fig(
        title="LIF1 Current dynamics", 
        x_axis_label='time (ms)', 
        y_axis_label='Current (U)',
        x=x, 
        y_arrays=current_lif1_y_arrays, 
        sizing_mode="stretch_both", 
        tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
        tooltips="Data point @x: @y",
        legend_location="top_right",
        legend_bg_fill_color="navy",
        legend_bg_fill_alpha=0.1,
        x_range=voltage_lif2_plot.x_range,    # Link the x-axis range to the voltage plot
    )

    # bplt.show(voltage_lif1_plot)

## Show the Plots assembled in a grid

In [115]:
import bokeh.plotting as bplt
from bokeh.layouts import gridplot

showPlot = True
if showPlot:
    # Create array of plots to be shown
    if not LIGHT_RUN:
        plots = [voltage_lif1_plot, current_lif1_plot, voltage_lif2_plot, current_lif2_plot]
    else:
        plots = [voltage_lif2_plot, current_lif2_plot]

    if len(plots) == 1:
        grid = plots[0]
    else:   # Create a grid layout
        grid = gridplot(plots, ncols=2, sizing_mode="stretch_both")

    # Show the plot
    bplt.show(grid)

## Export the plot to a file

In [103]:
export = False

OUT_FOLDER = "./results/net_burst"
OUT_FILENAME = f"lab_ch1-{n1}_{num_spikes_to_burst}spikes_all_ch_20ms_{num_steps}steps"

if export:
    file_path = f"{OUT_FOLDER}/{OUT_FILENAME}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="Network Burst detection - Voltage and Current dynamics")

    # Save the plot
    bplt.save(grid)

## Calculate Detection Metrics

### Convert the spike times to the same format as the Ground Truth

In [104]:
# Invert the mapping of the electrodes to the neuron indices
channel_map_inv = {v: k for k, v in channel_map.items()}
print("Channel Map Inverted: ", channel_map_inv)

Channel Map Inverted:  {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28, 28: 29, 29: 30, 30: 31, 31: 32, 32: 33, 33: 34, 34: 35, 35: 36, 36: 37, 37: 38, 38: 39, 39: 40, 40: 41, 41: 42, 42: 43, 43: 44, 44: 45, 45: 46, 46: 47, 47: 48, 48: 49, 49: 50, 50: 51, 51: 52, 52: 53, 53: 54, 54: 55, 55: 56, 56: 57, 57: 58, 58: 59, 59: 60, 60: 61, 61: 62, 62: 63, 63: 64, 64: 65, 65: 66, 66: 67, 67: 68, 68: 69, 69: 70, 70: 71, 71: 72, 72: 73, 73: 74, 74: 75, 75: 76, 76: 77, 77: 78, 78: 79, 79: 80, 80: 81, 81: 82, 82: 83, 83: 84, 84: 85, 85: 86, 86: 87, 87: 88, 88: 89, 89: 90, 90: 91, 91: 92, 92: 93, 93: 94, 94: 95, 95: 96, 96: 97, 97: 98, 98: 99, 99: 100, 100: 101, 101: 102, 102: 103, 103: 104, 104: 105, 105: 106, 106: 107, 107: 108, 108: 109, 109: 110, 110: 111, 111: 112, 112: 113, 113: 114, 114: 115, 115: 116, 116: 117, 117: 118, 118: 119, 119:

In [105]:
predicted_spikes = list( map(lambda x: x[0], spike_times_lif2 ) )
predicted_spikes = np.array(predicted_spikes)

print(f"Predicted {len(predicted_spikes)} spikes")
print("Predicted Spikes: ", predicted_spikes)

Predicted 36 spikes
Predicted Spikes:  [ 5012  5891  5944  6004 20757 ... 84978 86604 87507 87553 87602]


### Calculate the Confusion Matrix

In [106]:
# Initialize the variables that store the values of the Confusion Matrix
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

fp_timestamps = []

In [107]:
# Go through the ground truth and check if the predicted spikes are correct
# TODO: For now using I'm considering the causality window [gt_burst_time - CAUSALITY_WINDOW, gt_burst_time + CAUSALITY_WINDOW]. 

# Get the True Positive and False Negative values by comparing with the ground truth
for gt_net_burst_time in gt_cropped:
    # Check if the predicted burst time is within the causality window [gt_burst_time, gt_burst_time + CAUSALITY_WINDOW]
    if any([abs(pred_burst_time - gt_net_burst_time) <= NET_CAUSALITY_WINDOW for pred_burst_time in predicted_spikes]):
        true_positive += 1
    else:
        false_negative += 1

# Get the False Positive values by checking if the predicted spikes are false positives
for pred_net_burst_time in predicted_spikes:
    # Check if the predicted burst time is within the causality window [gt_burst_time, gt_burst_time + CAUSALITY_WINDOW]
        if all([abs(pred_net_burst_time - gt_net_burst_time) > NET_CAUSALITY_WINDOW for gt_net_burst_time in gt_cropped]):
            false_positive += 1

            # Add the false positive timestamp to the list
            fp_timestamps.append(pred_net_burst_time)

# Calculate the True Negative value
# TN = P - (TP + FP + FN)
true_negative = num_steps - true_positive - false_positive - false_negative

In [108]:
# Print the Confusion Matrix
print("True Positive: ", true_positive)
print("False Positive: ", false_positive)
print("True Negative: ", true_negative)
print("False Negative: ", false_negative)

# Print the Total of predictions
total_predictions = true_positive + false_positive + true_negative + false_negative
print("Total Predictions: ", total_predictions)

True Positive:  28
False Positive:  8
True Negative:  99964
False Negative:  0
Total Predictions:  100000


In [109]:
# Calculate relevant metrics
accuracy = 0
precision = 0
recall = 0
f1_score = 0
specificity = 0
if true_positive + false_positive == 0 == 0:
    print("No relevant predictions were made. Cannot calculate metrics.")
else:    
    accuracy = (true_positive + true_negative) / total_predictions * 100    # Proportion of correct predictions
    precision = true_positive / (true_positive + false_positive) * 100      # Proportion of TPs that were identified correctly
    recall = true_positive / (true_positive + false_negative) * 100         # Proportion of TPs that were captured by the model
    f1_score = (2 * precision * recall) / (precision + recall)              # Harmonic mean of Precision and Recall
    specificity = true_negative / (true_negative + false_positive) * 100    # Proportion of TNs that were identified correctly

print(f"Accuracy: {accuracy}%")
print(f"Precision: {precision}%")
print(f"Recall: {recall}%")
print(f"F1 Score: {f1_score}")
print(f"Specificity: {specificity}%")

Accuracy: 99.992%
Precision: 77.77777777777779%
Recall: 100.0%
F1 Score: 87.50000000000001
Specificity: 99.99199775937262%


### Print the timestamps with False Positives

In [110]:
print("False Positive Timestamps: ", fp_timestamps)

False Positive Timestamps:  [20757, 26743, 49918, 53930, 77353, 78230, 84978, 87602]


# Export the results of the Detection to a JSON file
Export the results of the classification to a JSON file. This file will include:
- `Causality Window` used
- Classification Metrics (`True Positives`, `False Positives`, `True Negatives`, `False Negatives`, `Accuracy`, `Precision`, `Recall`, `F1 Score`, `Specificity`)

In [113]:
import json

# Export the results to a JSON file

# Create a dictionary with the results
json_results = {
    "causality_window": NET_CAUSALITY_WINDOW,
    "metrics": {
        "true_positive": true_positive,
        "false_positive": false_positive,
        "true_negative": true_negative,
        "false_negative": false_negative,
        "total_predictions": total_predictions,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "specificity": specificity
    }
}

EXPORT_JSON_FILE = False
if EXPORT_JSON_FILE:
    json_file_name = f"{OUT_FOLDER}/{OUT_FILENAME}_metrics.json"
    with open(json_file_name, 'w') as f:
        json.dump(json_results, f)

## Stop the Runtime

In [112]:
selected_lif1.stop()